In [1]:
import os, sys
import numpy as np
from constants import GAS_CONST, PR_ATM
from constants import KCAL_JL, HT_JL
import pandas as pd
import math

In [2]:
cwd = os.getcwd()

In [3]:
def set_paths(myPath):
    """
    Set the absolute path to required files on the current machine.

    Returns
    -------
    reactionlist_path     : str
                            path to the file `complete_reactionlist.dat`
    rateconstantlist_path : str
                            path to the file `complete_rateconstantlist.dat`
    free_enenry_path      : str
                            path to the file `compositionlist.dat`
    """
    #myPath = os.path.dirname(os.path.abspath(__file__))
    reactionlist_path = myPath + '/data/complete_reaction_list.dat'
    rateconstantlist_path = myPath + '/data/complete_rateconstant_list.dat'
    free_energy_path = myPath + '/data/free_energy_library.dat'
    return reactionlist_path, rateconstantlist_path, free_energy_path

In [4]:
file_reactionlist, file_rateconstantlist, file_free_energy = set_paths(cwd)

In [5]:
print(file_rateconstantlist)

/Users/chowdhury/Documents/kmpy/examples/data/complete_rateconstant_list.dat


In [6]:
class Reaction(object):
    
    
    def __init__(self):
        self.reactants_names = []
        self.products_names = []
        self.uniqueSpeciesList = []
        #species_names = []
        
   
    def getReactantsName(self, line):
        
        for spec in line.split(','):
            if float(spec.split('_')[0].split()[0]) < 0:
                self.reactants_names.append((spec.split('_')[0].split()[0],
                                          spec.split('_')[1].split()[0]))
            #print(self.species_names)
        return self.reactants_names
    
    def getProductsName(self, line):
        
        for spec in line.split(','):
            if float(spec.split('_')[0].split()[0]) > 0:
                self.products_names.append((spec.split('_')[0].split()[0],
                                          spec.split('_')[1].split()[0]))
            #print(self.species_names)
        return self.products_names
    
    def uniqueSpeciesName(self, line, species_list):
        #self.uniqueSpeciesList = species_list
        for spec in line.split(','):
            #self.uniqueSpeciesList = species_list
            # If the species has already been added to the list then move on.
            if spec.split('_')[1].split()[0] in species_list:
                self.uniqueSpeciesList = species_list
                continue
            else:
                #print(self.uniqueSpeciesList)
                self.uniqueSpeciesList = species_list
                self.uniqueSpeciesList.append(spec.split('_')[1].split()[0])
            #print(spec.split('_')[1].split()[0])
        return self.uniqueSpeciesList
    

In [7]:
def build_species_list(reaction_file):
    """
    Build reactnat and product list for each reaction. Also builds a list
    of unique species in the mechanism
    Parameters
    ----------
    reaction_file       : str
                           path to the file `complete_reaction_list.dat`
    Returns
    __________

    reactant_list       : list
                         a list of the reactants and their stoichiometric
                         coeffs for each reaction
    product_list        : list
                         a list of the products and their stoichiometric
                         coeffs for each reaction
    species_list        : list
                        a list of unique species in the mechanism
    """

    #initializing reactant, product and unique species list
    reactant_list = []
    product_list = []
    species_name = []

    for line in open(reaction_file, 'r').readlines():
        reac = Reaction()
        reactant_list.append(reac.getReactantsName(line))
        product_list.append(reac.getProductsName(line))
        current_species = species_name
        #print(current_species)
        species_list = reac.uniqueSpeciesName(line, current_species)
        #print(species_name)
    species_list.sort()

    return reactant_list, product_list, species_list

reactants_list, products_list, unique_species = build_species_list(file_reactionlist)
    

In [8]:
print(len(unique_species))

46


In [9]:
print(len(products_list))


62


In [10]:
reac_prod_list = [react + prod for react, prod in zip(reactants_list, products_list)]
# for react, prod in zip(reactants, products):
#     total.append(react + prod)
#print(total)

In [11]:
reac_prod_list

[[('-1.0', 'O[C@H]1[C@H](O)CO[C@@H](O)[C@@H]1O'),
  ('1.0', 'O'),
  ('1.0', 'OC1=COC[C@@H](O)[C@@H]1O')],
 [('-1.0', 'O[C@H]1[C@H](O)CO[C@@H](O)[C@@H]1O'),
  ('1.0', 'O'),
  ('1.0', 'OC1=C[C@@H](O)[C@H](O)OC1')],
 [('-1.0', 'O[C@H]1[C@H](O)CO[C@@H](O)[C@@H]1O'),
  ('1.0', 'O'),
  ('1.0', 'O[C@H]1C=CO[C@@H](O)[C@@H]1O')],
 [('-1.0', 'O[C@H]1[C@H](O)CO[C@@H](O)[C@@H]1O'),
  ('1.0', 'O=C[C@@H](O)[C@H](O)[C@@H](O)CO')],
 [('-1.0', 'O[C@H]1C=CO[C@@H](O)[C@@H]1O'),
  ('1.0', 'O=C[C@H](O)[C@@H](O)/C=C\\O')],
 [('-1.0', 'O=C[C@H](O)[C@@H](O)/C=C\\O'),
  ('1.0', 'O=CC[C@H](O)[C@@H](O)C=O')],
 [('-1.0', 'O=CC[C@H](O)[C@@H](O)C=O'), ('1.0', 'O=CCO'), ('1.0', 'O=CCC=O')],
 [('-1.0', 'O=CC[C@H](O)[C@@H](O)C=O'),
  ('1.0', 'O'),
  ('1.0', 'O=C/C=C\\[C@@H](O)C=O')],
 [('-1.0', 'O[C@H]1C=CO[C@@H](O)[C@@H]1O'),
  ('1.0', 'O'),
  ('1.0', 'OC1=COC=C[C@@H]1O')],
 [('-1.0', 'OC1=C[C@@H](O)[C@H](O)OC1'), ('1.0', 'O=C[C@H](O)/C=C(/O)CO')],
 [('-1.0', 'O=C[C@H](O)/C=C(/O)CO'), ('1.0', 'O=C[C@H](O)CC(=O)CO')],

In [12]:
#generating a dictionary of unique species from the species_list
speciesindices = {unique_species[i]: i for i in range(0, len(unique_species))}

indices_to_species = dict(zip(speciesindices.values(), speciesindices.keys()))

In [13]:
speciesindices

{'C(=O)=O': 0,
 'C-atom': 1,
 'C=C(O)C=O': 2,
 'C=C(O)[C@@H](O)[C@H](O)C=O': 3,
 'C=O': 4,
 'CC(=O)/C=C(/O)C=O': 5,
 'CC(=O)C=O': 6,
 'CC(=O)CC(=O)C=O': 7,
 'CC(=O)CO': 8,
 'CC(=O)[C@@H](O)[C@H](O)C=O': 9,
 'CC=O': 10,
 'O': 11,
 'O/C=C/O': 12,
 'O=C/C=C(\\O)[C@@H](O)CO': 13,
 'O=C/C=C/C(=O)CO': 14,
 'O=C/C=C\\[C@@H](O)C=O': 15,
 'O=C1COC=C[C@@H]1O': 16,
 'O=C1COCC(=O)C1': 17,
 'O=C1COC[C@@H](O)[C@@H]1O': 18,
 'O=CC(O)C=O': 19,
 'O=CC1=C[C@@H](O)CO1': 20,
 'O=CC=O': 21,
 'O=CCC(=O)[C@@H](O)CO': 22,
 'O=CCC=O': 23,
 'O=CCO': 24,
 'O=CC[C@H](O)[C@@H](O)C=O': 25,
 'O=CC[C@H](O)[C@H](O)C=O': 26,
 'O=C[C@@H](O)/C=C(\\O)CO': 27,
 'O=C[C@@H](O)CC(=O)CO': 28,
 'O=C[C@@H](O)[C@@H](O)/C=C\\O': 29,
 'O=C[C@@H](O)[C@@H](O)CO': 30,
 'O=C[C@@H](O)[C@H](O)[C@@H](O)CO': 31,
 'O=C[C@H](O)/C=C(/O)CO': 32,
 'O=C[C@H](O)CC(=O)CO': 33,
 'O=C[C@H](O)CO': 34,
 'O=C[C@H](O)[C@@H](O)/C=C\\O': 35,
 'O=C[C@H]1OC[C@H](O)[C@H]1O': 36,
 'O=Cc1ccco1': 37,
 'OC1=COC=C[C@@H]1O': 38,
 'OC1=COCC(O)=C1': 39,
 'OC1=COC[C@

In [14]:
class Kinetic_params(object):
      
    def __init__(self):
        self.forward_rate_params = []
        self.forward_rates = []
        #self.forward_E = []
        #self.uniqueSpeciesList = []
        #species_names = []
    
    def getForwardRateParameters(self, line):
        
        self.forward_rate_params = [line.split(' ')[0], line.split(' ')[1],
                      line.split(' ')[2].split()[0]]
      
        return self.forward_rate_params
    
    def getForwardRateConstant(self, parameters, T):
        
        self.forward_rates = eval(params[0]) * np.exp(- eval(params[2]) * KCAL_JL /
                                                               (GAS_CONST * temp))
        return self.forward_rates

In [15]:
forward_rate_constants = []
temp = 573


In [16]:
for line in open(file_rateconstantlist, 'r').readlines():
    f_params = Kinetic_params()
    params = f_params.getForwardRateParameters(line)
    forward_rate_constants.append(f_params.getForwardRateConstant(params, temp))

In [17]:
forward_rate_constants

[9.070586400466592e-07,
 9.070586400466592e-07,
 1.8791556907858517e-07,
 3.548590208923674e-05,
 2.346720330517098e-05,
 1.410002815679657e-05,
 0.0005585146932428701,
 3.281645112867024e-05,
 9.070586400466592e-07,
 2.346720330517098e-05,
 1.410002815679657e-05,
 3.281645112867024e-05,
 0.0005585146932428701,
 9.070586400466592e-07,
 1.410002815679657e-05,
 9.070586400466592e-07,
 1.410002815679657e-05,
 2.346720330517098e-05,
 1.410002815679657e-05,
 0.0005760247230552326,
 3.281645112867024e-05,
 1.410002815679657e-05,
 1.8791556907858517e-07,
 1.410002815679657e-05,
 3.281645112867024e-05,
 0.01126663897467137,
 1.407177318117666e-06,
 3.281645112867024e-05,
 1.410002815679657e-05,
 0.0005585146932428701,
 3.281645112867024e-05,
 1.410002815679657e-05,
 0.0005585146932428701,
 9.070586400466592e-07,
 3.281645112867024e-05,
 1.410002815679657e-05,
 0.0005585146932428701,
 0.0005585146932428701,
 0.008681988741437899,
 0.0005760247230552326,
 0.0005585146932428701,
 9.07058640046659

In [23]:
def build_free_energy_dict(free_energy_path, T):
    """
    Build a dictionary of free energy at a given temperature for all the species 
    present in the mechanism. It reads the file free_energy_path which is basically
    a library of gibbs free energy correction at different molecules at different temperatures.
    Parameters
    ----------
    completereactionlist : str
                           path to the file `free_energy_library.dat`
    T                    : float
                           temperature to calculate free energy
    Returns
    -------
    free_energy.    : dict
                    a dictionary where keys are unique species and values
                    are free energy of species at a given temperature 
                    build from free_energy_library.dat
    """
    
    df = pd.read_csv(free_energy_path, sep = '\t')

    if "{}K".format(T) in df.columns:
        df["Free Energy @{}K".format(T)] = df['Zero_point'] + df["{}K".format(T)]
    else:
        temp_low = math.floor(T / 100.0)* 100
        temp_high = math.ceil(T / 100.0)* 100
        df["{}K".format(T)] = (df["{}K".format(temp_high)] - df["{}K".format(temp_high)]*(temp - temp_low)/(temp_high - temp_low) + df["{}K".format(temp_low)])
        df["Free Energy @{}K".format(T)] = df['Zero_point'] + df["{}K".format(T)]
    print(df.head())
    
    free_energy = dict([(i, a) for i, a  in zip(df.smiles, df['Free Energy @' + str(T) +'K'])])
    
    return free_energy

In [25]:
free_energy_dict = build_free_energy_dict(file_free_energy, temp)
print(len(free_energy_dict))

   Structure ID                              smiles  Zero_point      300K  \
0             1  O[C@H]1[C@H](O)CO[C@@H](O)[C@@H]1O -572.654636  0.130831   
1             2                                   O  -76.422977  0.003836   
2             3            OC1=COC[C@@H](O)[C@@H]1O -496.193201  0.103466   
3             4           OC1=C[C@@H](O)[C@H](O)OC1 -496.205201  0.104026   
4             5        O[C@H]1C=CO[C@@H](O)[C@@H]1O -496.204142  0.104043   

       400K      500K      600K      700K      800K      573K  \
0  0.114449  0.096034  0.075704  0.053580  0.029778  0.116474   
1 -0.003547 -0.011258 -0.019238 -0.027448 -0.035861 -0.016452   
2  0.088156  0.071087  0.052368  0.032109  0.010414  0.085226   
3  0.088926  0.072080  0.053591  0.033566  0.012107  0.086550   
4  0.088886  0.071984  0.053439  0.033355  0.011835  0.086413   

   Free Energy @573K  
0        -572.538162  
1         -76.439430  
2        -496.107975  
3        -496.118652  
4        -496.117729  
46


In [20]:
def build_reverse_rate_constants(complete_list, free_energy, forward_rates, T):
    """
    Calculate the reverse rate constants of all the reactions. There are two steps 
    doing this
    1. Calculate the change in free energy for each reaction
            delG = G(products) - G(reactanat)
    This is calculated from the complete lists of reactions and free_energy_dict
    2. Use delG to calculate the equlilibrium constant
            Keq = exp (- delG/Gas Const * Temp)
    3. Use the following equation to calculate the reverse rate constant
            Keq = Kf / Kr * (Gas Const * Temp / Pressure)^n
    where n = total number of product molecules - total number of reactant molecules
    ----------
    complete_list        : list
                           A list of all the reactions with reactant and 
                         product species and their stoichimetric coeffs
    free_energy          : dict
                          A dictionary of free energies of all the species
                          at a given temperature, obtained from 
                          build_free_energy_dict function
    forward_rate         : A list of forward rate constants for all the reactions
                         obtained from build_forward_reaction_rates
    T                    : float
                           temperature to calculate free energy
    Returns
    -------
     reverse_rates       : list
                         A list of reverse rate constants
    """
    
    
    mol_change = []
    gibbs_energy_list = []

    for i, item in enumerate (complete_list):
        n_reac = 0
        n_prod = 0
        reac_free_energy = 0
        prod_free_energy = 0
        for entry in item:
        
            if float(entry[0]) < 0:
                n_reac = n_reac + abs(float(entry[0]))
                reac_free_energy = free_energy[entry[1]] + abs(float(entry[0])) * reac_free_energy
            else:
                prod_free_energy = free_energy[entry[1]] + abs(float(entry[0])) * prod_free_energy
                n_prod = n_prod + abs(float(entry[0]))
        #print(n_reac)
        mol_change.append(n_prod - n_reac)
        #print(mol_change)
        gibbs_energy_list.append((prod_free_energy - reac_free_energy) * HT_JL)
    
    equilibrium_constants = [np.exp(-n*1000/(GAS_CONST * T)) for n in gibbs_energy_list]
    reverse_rates = [(a / b)* 1000 * (GAS_CONST * T / PR_ATM)**c if c < 3 else 0 for (a, b, c) in zip(forward_rates, equilibrium_constants, mol_change)]
    
    return reverse_rates
    

In [21]:
reverse_rate_constants = build_reverse_rate_constants(reac_prod_list, free_energy_dict, forward_rate_constants, temp)
print(reverse_rate_constants)

[2.744108790974956e-07, 7.636563061286407e-10, 2.6349264591436553e-10, 0.35862533918796063, 1.0548342765630527, 3.6641644819074258e-06, 1.0894949484310647e-08, 3.3699787181501417e-07, 7.681395428784125e-09, 0.4893722934922695, 1.7252967599947549e-06, 8.040955970886511e-08, 1.8353307092164633e-07, 2.6953490046390926e-08, 1.4530460286212018e-09, 7.500869759171973e-11, 4.912027399630005e-06, 0.0019782365229708628, 9.772778606340298e-08, 1.5108391675991245e-06, 4.6550110726307026e-10, 0.001198962944235727, 7.375768819740329e-12, 0.014623317422506596, 2.2087262363454944e-08, 3.4115412482360156e-06, 9.549599501963518e-17, 2.2032795639310154e-08, 5.548475505886963e-06, 3.5381704075358e-08, 1.3398150007052011e-09, 1.4601785519383102e-05, 5.976187373523655e-06, 0.0009515258237794512, 6.929917095901439e-11, 0.0013937393372645027, 8.405321830456699e-08, 1.8183058631060237e-07, 1.1392260311833235e-08, 6.1021106088528395e-05, 4.524194483364314e-05, 1.441824886686969e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
def build_reac_prod_dict(reac_list, prod_list, speciesindices):
    """
    Build a dictionary of the reactants involved in each reaction,
    along with their stoichiometric coefficients.  The keys of the
    dictionary are the reaction numbers, the values are lists of lists
    [[reactant1index, -1*coeff1],...]
    Parameters
    ----------
    completereactionlist : str
                           path to the file `complete_reaction_list.dat`
    speciesindices       : dict
                           the dictionary speciesindices from
                           get_speciesindices()
    Returns
    -------
    reactant_dict : dict
                    a dictionary where keys are reaction numbers and values
                    are lists of lists with the reactants and their
                    stoichiometric coefficients for each reaction
    """
    reactant_dict = {}
    for rxnindex, reaction in enumerate(reac_list):
        reactants = []
        #
        for x in range(len(reaction)):
            # if the species is a reactant
         #   if float(x.split('_')[0]) < 0:
            reactants.append([speciesindices[reaction[x][1]],
                                -1*float(reaction[x][0])])
            #    in preceding line: *-1 because I want the |stoich coeff|
        reactant_dict[rxnindex] = reactants
        
    products_dict = {}
    for rxnindex, reaction in enumerate(prod_list):
        products = []
        #
        for x in range(len(reaction)):
            # if the species is a reactant
         #   if float(x.split('_')[0]) < 0:
            products.append([speciesindices[reaction[x][1]],
                                1*float(reaction[x][0])])
            #    in preceding line: *-1 because I want the |stoich coeff|
        products_dict[rxnindex] = products
    return reactant_dict, products_dict

In [ ]:
reac_dict, prod_dict = build_reac_prod_dict(reactants_list, products_list, speciesindices)
prod_dict


In [ ]:
def build_reac_species_dict(reacprodlist, specieslist):
    """
    Build a dictionary where keys are species and values are lists with the
    reactions that species is involved in, that reaction's sign in the net
    rate equation, and the stoichiometric coefficient of the species in that
    reaction.
    Parameters
    ----------
    reacprodlist : list
                        a list of both reactants and products and their 
                        stoichiometric co-effs
    specieslist  : list
                        a list of unique species in the mecahnism
    
    Returns
    -------
    reac_species : dict
                   keys are the species in the model; values are lists of
                   [reaction that species is involved in,
                   sign of that species in the net rate equation,
                   stoichiometric coefficient]
    """
    #specieslist = get_specieslist(set_paths()[0])
    reac_species = {}
    for species in specieslist:
        # This loop makes a list of which reactions "species" takes part in
        # and what sign that term in the net rate eqn has
        # and what the stoichiometric coefficient is
    
        reactions_involved = []
        for rxnindex, reac_list in enumerate (reacprodlist):
            for x in range(len(reac_list)):
                # If the species being iterated over is part of this reaction
                if species == reac_list[x][1]:
                    # if the species is a reactant
                    if float(reac_list[x][0]) < 0:
                        reactions_involved.append(
                            [rxnindex, -1, str(-1), '+'+str(1)])
                    
                    # if the species is a product
                    if float(reac_list[x][0]) > 0:
                        reactions_involved.append(
                            [rxnindex, 1, '+'+str(1), str(-1)])
    
        reac_species[species] = reactions_involved
    return reac_species

In [ ]:
reac_species = build_reac_species_dict(reac_prod_list, unique_species)
reac_species

In [ ]:
def build_rate_eqn(k_mat, r_dict, s_indices, human, forward):

    """ This function writes the list of rate expressions for each reaction.
    Parameters
    ----------
    kmat               : list
                         A list of reaction rate contstants (k_forward or k_reverse)
    r_dict             : dictionary
                         reactant or product directory
    s_indices          : dict
                         the reverse of speciesindices (keys are the indices
                         and values are the species)
    human              : str, optional
                         indicate whether the output of this function should
                         be formatted for a human to read ('yes'). Default
                         is 'no'
    forward             : str
                        reaction type,if 'yes', it is forward reaction
                        default is 'yes'
    Returns
    -------
    rates_list : list
                A list of the rate expressions for all the reactions in the mecahnism
    """

    rates_list = []
    for i, line in enumerate(k_mat):
        if forward == 'yes':
            rate = 'rate_f[%s] = kf(T,%s) ' % (i, i)
        else:
            rate = 'rate_r[%s] = kr(T,%s) ' % (i, i)
        concentrations = ''
        for entry in r_dict[i]:
            if entry == 'n':   # if there is no reaction
                concentrations = '* 0'
                break
            else:
                if human == 'no':
                    concentrations += '* y[%s]**%s ' % (entry[0], entry[1])
                elif human == 'yes':
                    concentrations += '* [%s]**%s ' % \
                        (s_indices[entry[0]], entry[1])
                else:
                    raise ValueError('human must be a string: yes or no')
        rate += concentrations

        #rate = rate_reactant + rate_product
        rates_list.append(rate)
        
    return rates_list


rates_f = build_rate_eqn(forward_rate_constants, reac_dict, indices_to_species, human = 'no', forward = 'yes')
rates_r = build_rate_eqn(reverse_rate_constants, prod_dict, indices_to_species, human = 'no', forward = 'no')


print(rates_r)

In [ ]:
def build_dydt_list(rates_forward, rates_reverse, specieslist, species_rxns, human='no'):
    """This function returns the list of dydt expressions generated for all
    the reactions from rates_list.
    Parameters
    ----------
    rates_list         : list
                         the output of build_rates_list()
    specieslist        : list
                         a list of all the species in the kinetic scheme
    species_rxns       : dict
                         dictionary where keys that are the model species and
                         values are the reactions they are involved in
    human              : str, optional
                         indicate whether the output of this function should
                         be formatted for a human to read ('yes'). Default
                         is 'no'
    Returns
    -------
    dydt_expressions : list
                       expressions for the ODEs expressing the concentration
                       of each species with time
    """
    dydt_expressions = []
    for species in specieslist:
        rate_formation = 'd[%s]/dt = ' % (species)
        # "entry" is [reaction#, sign of that reaction, coefficient]
        for entry in species_rxns[species]:
            if human == 'no':
                rate_formation += '%s*%s' % \
                    (entry[2], rates_forward[entry[0]].split(' = ')[1])
                rate_formation += '%s*%s' % \
                    (entry[3], rates_reverse[entry[0]].split(' = ')[1])
            elif human == 'yes':
                rate_formation += '%s*rate_f[%s] ' % (entry[2], entry[0])
                rate_formation += '%s*rate_r[%s] ' % (entry[3], entry[0])
            else:
                raise ValueError('human must be a string: yes or no')
        dydt_expressions.append(rate_formation)
    return dydt_expressions


In [ ]:
dydt_list = build_dydt_list(rates_f, rates_r, unique_species, reac_species, human='yes')

In [ ]:
dydt_list

In [ ]:
temperature = [300, 373, 473, 573, 673, 773]
import matplotlib.pyplot as plt

for temp in temperature:
    df = pd.read_csv(file_free_energy, sep = '\t')
    
    if str(temp)+'K' in df.columns:
        df['Free Energy @' + str(temp) +'K'] = df['Zero_point'] + df[str(temp)+'K']
    else:
        temp_low = math.floor(temp / 100.0)* 100
        temp_high = math.ceil(temp / 100.0)* 100
        df[str(temp)+'K'] = (df[str(temp_high)+'K'] - df[str(temp_low)+'K'])*(temp - temp_low)/(temp_high - temp_low) + df[str(temp_low)+'K']
        df['Free Energy @' + str(temp) +'K'] = df['Zero_point'] + df[str(temp)+'K']
    
    free_energy_dict = dict([(i, a) for i, a  in zip(df.smiles, df['Free Energy @' + str(temp) +'K'])])
    
    free_energy_change = []

    for i, item in enumerate (reac_prod_list):
        n_reac = 0
        n_prod = 0
        reac_free_energy = 0
        prod_free_energy = 0
        for entry in item:
        
            #print(entry[0])
            if float(entry[0]) < 0:
                n_reac = n_reac + abs(float(entry[0]))
                reac_free_energy = free_energy_dict[entry[1]] + reac_free_energy
            else:
                prod_free_energy = free_energy_dict[entry[1]] + prod_free_energy
                n_prod = n_prod + abs(float(entry[0]))
        #print(n_reac)
        mol_change.append(n_prod - n_reac)
        #print(mol_change)
        free_energy_change.append((prod_free_energy - reac_free_energy) * 2625.5)
    plt.plot(free_energy_change, label = '%sK' %temp)

plt.legend()
plt.xlim(-2,10)
plt.ylim(-80,50)
plt.show()
